# Προχωρημένα Θέματα Βάσεων Δεδομένων

**Ονοματεπώνυμο:** Κωνσταντίνος Διβριώτης

**ΑΜ:** 03114140

## Query 3: 

Χρησιμοποιώντας ως αναφορά τα δεδομένα της απογραφής 2010 για τον πληθυσμό και εκείνα της απογραφής του 2015 για το εισόδημα ανα νοικοκυριό, να υπολογίσετε για κάθε περιοχή του Los Angeles τα παρακάτω:
- Το μέσο ετήσιο εισόδημα ανά άτομο
- Την αναλογία συνολικού αριθμού εγκλημάτων ανά άτομο

In [1]:
from pyspark.sql import SparkSession
from sedona.spark import *

spark = SparkSession \
    .builder \
    .appName("CensusDataAnalysis") \
    .getOrCreate()

sedona = SedonaContext.create(spark)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2499,application_1732639283265_2458,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
DATA_BUCKET = "s3://initial-notebook-data-bucket-dblab-905418150721"
GROUP_BUCKET = "s3://groups-bucket-dblab-905418150721/group15"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Διάβασμα και Επισκόπηση αρχείων εισόδου

In [3]:
from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql.functions import regexp_replace, col

income_schema = StructType([
    StructField("Zip Code", StringType()),
    StructField("Community", StringType()),
    StructField("Estimated Median Income", StringType())
])

income_data = spark.read.csv(f"{DATA_BUCKET}/LA_income_2015.csv", header=True, schema=income_schema)

# Μετατροπή του Estimated Median Income σε αριθμητική μορφή
income_data = income_data \
    .withColumn(
        "Estimated Median Income",
        regexp_replace(col("Estimated Median Income"), "[$,]", "").cast("float")
    ) \
    .select("Zip Code", "Estimated Median Income")

income_data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------------+
|Zip Code|Estimated Median Income|
+--------+-----------------------+
|   90001|                33887.0|
|   90002|                30413.0|
|   90003|                30805.0|
|   90004|                40612.0|
|   90005|                31142.0|
+--------+-----------------------+
only showing top 5 rows

In [4]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType

# Ορισμός του schema των dataset
crimes_schema = StructType([
    StructField("DR_NO", StringType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", StringType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", StringType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType()),
    StructField("Premis Cd", StringType()),
    StructField("Premis Desc", StringType()),
    StructField("Weapon Used Cd", IntegerType()),
    StructField("Weapon Desc", StringType()),
    StructField("Status", StringType()),
    StructField("Status Desc", StringType()),
    StructField("Crm Cd 1", IntegerType()),
    StructField("Crm Cd 2", IntegerType()),
    StructField("Crm Cd 3", IntegerType()),
    StructField("Crm Cd 4", IntegerType()),
    StructField("LOCATION", StringType()),
    StructField("Cross Street", StringType()),
    StructField("LAT", DoubleType()),
    StructField("LON", DoubleType())
])

# Διαβάζουμε τα 2 datasets (2010-2019 και 2020-σήμερα) και τα συνενώνουμε σε 1
crime_data_2010_2019 = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, schema=crimes_schema)
crime_data_2020_present = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, schema=crimes_schema)
crime_data = crime_data_2010_2019.union(crime_data_2020_present)

# Μετατρέπουμε τις στήλες LAT, LON σε geometry με το ST_POINT
crime_data = crime_data \
                .withColumn("geom", ST_Point("LON", "LAT")) \
                .filter(col("geom") != ST_Point(0, 0)) \
                .select("DR_NO", "geom")

crime_data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|    DR_NO|                geom|
+---------+--------------------+
|001307355|POINT (-118.2695 ...|
|011401303|POINT (-118.3962 ...|
|070309629|POINT (-118.2524 ...|
|090631215|POINT (-118.3295 ...|
|100100501|POINT (-118.2488 ...|
+---------+--------------------+
only showing top 5 rows

In [5]:
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true") \
            .load(f"{DATA_BUCKET}/2010_Census_Blocks.geojson") \
            .selectExpr("explode(features) as features") \
            .select("features.*")

blocks_data = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                    blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type") \
            .filter(
                col("COMM").isNotNull() & (col("CITY") == "Los Angeles") &
                (col("HOUSING10") > 0) & (col("POP_2010") > 0)
            ) \
            .select("COMM", "ZCTA10", "HOUSING10", "POP_2010", "geometry")

blocks_data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+---------+--------+--------------------+
|     COMM|ZCTA10|HOUSING10|POP_2010|            geometry|
+---------+------+---------+--------+--------------------+
|San Pedro| 90732|       26|      69|POLYGON ((-118.31...|
|San Pedro| 90731|       70|     120|POLYGON ((-118.28...|
|San Pedro| 90731|       86|     240|POLYGON ((-118.29...|
|San Pedro| 90732|       29|      75|POLYGON ((-118.31...|
|San Pedro| 90731|       80|     246|POLYGON ((-118.28...|
+---------+------+---------+--------+--------------------+
only showing top 5 rows

In [6]:
blocks_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- COMM: string (nullable = true)
 |-- ZCTA10: string (nullable = true)
 |-- HOUSING10: long (nullable = true)
 |-- POP_2010: long (nullable = true)
 |-- geometry: geometry (nullable = true)

In [7]:
blocks_data_description_schema = StructType([
    StructField("field", StringType()),
    StructField("type", StringType()),
    StructField("meaning", StringType())
])

blocks_data_description = spark.read.csv(f"{DATA_BUCKET}/2010_Census_Blocks_fields.csv", header=True, schema=blocks_data_description_schema)

blocks_data_description \
        .filter(col("field").isin("COMM", "ZCTA10", "HOUSING10", "POP_2010", "geometry")) \
        .show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+-------------------------------------------------------------------------+
|field    |type    |meaning                                                                  |
+---------+--------+-------------------------------------------------------------------------+
|COMM     |string  |Unincorporated area community name and LA City neighborhood              |
|HOUSING10|long    |2010 housing (PL 94-171 Redistricting Data Summary File - Total Housing) |
|POP_2010 |long    |Population (PL 94-171 Redistricting Data Summary File - Total Population)|
|ZCTA10   |string  |Zip Code Tabulation Area                                                 |
|geometry |geometry|Geometry of the block                                                    |
+---------+--------+-------------------------------------------------------------------------+

## Υλοποίηση με DataFrame

In [8]:
from pyspark.sql.functions import sum, count, col, max
import time

start_time = time.time()

# Για κάθε περιοχή του στο Census, υπάρχουν πολλαπλές
# εγγραφές για COMM, ZCTA10. Συνεπώς, πριν κάνουμε το
# join με το Income, πρέπει να κάνουμε group by για να
# υπολογίσουμε το συνολικό πληθυσμό ανά COMM, ZIP Code
result = blocks_data \
            .groupBy("COMM", "ZCTA10") \
            .agg(
                sum("POP_2010").alias("Population"),
                sum("HOUSING10").alias("Households"),
                ST_Union_Aggr("geometry").alias("geometry")
            )

# Έπειτα κάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code. Κάνουμε group by με το
# COMM για να βρούμε το συνολικό εισόδημα και το
# συνολικό πληθυσμό ανά COMM.
result = result \
            .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            )

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = result \
            .join(crime_data, ST_Within(crime_data["geom"], result["geometry"]), "inner") \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .orderBy("COMM")

result \
    .select("COMM", "Income per Person", "Crimes per Person") \
    .show(truncate=False)

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------------------+-------------------+
|COMM                   |Income per Person |Crimes per Person  |
+-----------------------+------------------+-------------------+
|Adams-Normandie        |8791.458301453711 |0.7148686559551135 |
|Alsace                 |11239.50136425648 |0.5416098226466576 |
|Angeles National Forest|33079.58823529412 |0.4117647058823529 |
|Angelino Heights       |18427.059814658805|0.5732940185341197 |
|Arleta                 |12110.779474388612|0.4264509064363061 |
|Atwater Village        |28481.236967160792|0.5288318320448259 |
|Baldwin Hills          |17303.906408241663|0.9950061114021302 |
|Bel Air                |63041.33942621959 |0.39922527539038855|
|Beverly Crest          |60947.48978754819 |0.3689607087195472 |
|Beverlywood            |29267.82118405155 |0.5084977849375755 |
|Boyle Heights          |8494.108286861341 |0.6171887393378809 |
|Brentwood              |60846.854461055365|0.4058638814936173 |
|Brookside              |

In [9]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 32.56 seconds

Θα δοκιμάσουμε να αναγκάσουμε το Spark να χρησιμοποιήσει διαφορετικές στρατηγικές, ώστε να συγκρίνουμε την απόδοσή τους.

### 1. BROADCAST

In [13]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Για κάθε περιοχή του στο Census, υπάρχουν πολλαπλές
# εγγραφές για COMM, ZCTA10. Συνεπώς, πριν κάνουμε το
# join με το Income, πρέπει να κάνουμε group by για να
# υπολογίσουμε το συνολικό πληθυσμό ανά COMM, ZIP Code
result = blocks_data \
            .groupBy("COMM", "ZCTA10") \
            .agg(
                sum("POP_2010").alias("Population"),
                sum("HOUSING10").alias("Households"),
                ST_Union_Aggr("geometry").alias("geometry")
            )

# Έπειτα κάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code. Κάνουμε group by με το
# COMM για να βρούμε το συνολικό εισόδημα και το
# συνολικό πληθυσμό ανά COMM.
result = result \
            .hint("BROADCAST") \
            .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            )

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = result \
            .hint("BROADCAST") \
            .join(crime_data, ST_Within(crime_data["geom"], result["geometry"]), "inner") \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .orderBy("COMM")

result \
    .select("COMM", "Income per Person", "Crimes per Person") \
    .show(truncate=False)

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------------------+-------------------+
|COMM                   |Income per Person |Crimes per Person  |
+-----------------------+------------------+-------------------+
|Adams-Normandie        |8791.458301453711 |0.7148686559551135 |
|Alsace                 |11239.50136425648 |0.5416098226466576 |
|Angeles National Forest|33079.58823529412 |0.4117647058823529 |
|Angelino Heights       |18427.059814658805|0.5732940185341197 |
|Arleta                 |12110.779474388612|0.4264509064363061 |
|Atwater Village        |28481.236967160792|0.5288318320448259 |
|Baldwin Hills          |17303.906408241663|0.9950061114021302 |
|Bel Air                |63041.33942621959 |0.39922527539038855|
|Beverly Crest          |60947.48978754819 |0.3689607087195472 |
|Beverlywood            |29267.82118405155 |0.5084977849375755 |
|Boyle Heights          |8494.108286861341 |0.6171887393378809 |
|Brentwood              |60846.854461055365|0.4058638814936173 |
|Brookside              |

In [14]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 26.90 seconds

In [15]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (35)
+- Sort (34)
   +- Exchange (33)
      +- Project (32)
         +- HashAggregate (31)
            +- Exchange (30)
               +- HashAggregate (29)
                  +- Project (28)
                     +- BroadcastIndexJoin (27)
                        :- SpatialIndex (19)
                        :  +- Filter (18)
                        :     +- ObjectHashAggregate (17)
                        :        +- Exchange (16)
                        :           +- ObjectHashAggregate (15)
                        :              +- Project (14)
                        :                 +- BroadcastHashJoin Inner BuildLeft (13)
                        :                    :- BroadcastExchange (9)
                        :                    :  +- ObjectHashAggregate (8)
                        :                    :     +- Exchange (7)
                        :                    :        +- ObjectHashAggregate (6)
                        :       

### 2. MERGE

In [16]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Για κάθε περιοχή του στο Census, υπάρχουν πολλαπλές
# εγγραφές για COMM, ZCTA10. Συνεπώς, πριν κάνουμε το
# join με το Income, πρέπει να κάνουμε group by για να
# υπολογίσουμε το συνολικό πληθυσμό ανά COMM, ZIP Code
result = blocks_data \
            .groupBy("COMM", "ZCTA10") \
            .agg(
                sum("POP_2010").alias("Population"),
                sum("HOUSING10").alias("Households"),
                ST_Union_Aggr("geometry").alias("geometry")
            )

# Έπειτα κάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code. Κάνουμε group by με το
# COMM για να βρούμε το συνολικό εισόδημα και το
# συνολικό πληθυσμό ανά COMM.
result = result \
            .hint("MERGE") \
            .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            )

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = result \
            .hint("MERGE") \
            .join(crime_data, ST_Within(crime_data["geom"], result["geometry"]), "inner") \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .orderBy("COMM")

result \
    .select("COMM", "Income per Person", "Crimes per Person") \
    .show(truncate=False)

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------------------+-------------------+
|COMM                   |Income per Person |Crimes per Person  |
+-----------------------+------------------+-------------------+
|Adams-Normandie        |8791.458301453711 |0.7148686559551135 |
|Alsace                 |11239.50136425648 |0.5416098226466576 |
|Angeles National Forest|33079.58823529412 |0.4117647058823529 |
|Angelino Heights       |18427.059814658805|0.5732940185341197 |
|Arleta                 |12110.779474388612|0.4264509064363061 |
|Atwater Village        |28481.236967160792|0.5288318320448259 |
|Baldwin Hills          |17303.906408241663|0.9950061114021302 |
|Bel Air                |63041.33942621959 |0.39922527539038855|
|Beverly Crest          |60947.48978754819 |0.3689607087195472 |
|Beverlywood            |29267.82118405155 |0.5084977849375755 |
|Boyle Heights          |8494.108286861341 |0.6171887393378809 |
|Brentwood              |60846.854461055365|0.4058638814936173 |
|Brookside              |

In [17]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 25.10 seconds

In [18]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (37)
+- Sort (36)
   +- Exchange (35)
      +- Project (34)
         +- HashAggregate (33)
            +- Exchange (32)
               +- HashAggregate (31)
                  +- Project (30)
                     +- RangeJoin (29)
                        :- Filter (21)
                        :  +- ObjectHashAggregate (20)
                        :     +- Exchange (19)
                        :        +- ObjectHashAggregate (18)
                        :           +- Project (17)
                        :              +- SortMergeJoin Inner (16)
                        :                 :- Sort (10)
                        :                 :  +- Exchange (9)
                        :                 :     +- ObjectHashAggregate (8)
                        :                 :        +- Exchange (7)
                        :                 :           +- ObjectHashAggregate (6)
                        :                 :              +- Project (5)


### 3. SHUFFLE HASH

In [19]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Για κάθε περιοχή του στο Census, υπάρχουν πολλαπλές
# εγγραφές για COMM, ZCTA10. Συνεπώς, πριν κάνουμε το
# join με το Income, πρέπει να κάνουμε group by για να
# υπολογίσουμε το συνολικό πληθυσμό ανά COMM, ZIP Code
result = blocks_data \
            .groupBy("COMM", "ZCTA10") \
            .agg(
                sum("POP_2010").alias("Population"),
                sum("HOUSING10").alias("Households"),
                ST_Union_Aggr("geometry").alias("geometry")
            )

# Έπειτα κάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code. Κάνουμε group by με το
# COMM για να βρούμε το συνολικό εισόδημα και το
# συνολικό πληθυσμό ανά COMM.
result = result \
            .hint("SHUFFLE_HASH") \
            .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            )

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = result \
            .hint("SHUFFLE_HASH") \
            .join(crime_data, ST_Within(crime_data["geom"], result["geometry"]), "inner") \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .orderBy("COMM")

result \
    .select("COMM", "Income per Person", "Crimes per Person") \
    .show(truncate=False)

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------------------+-------------------+
|COMM                   |Income per Person |Crimes per Person  |
+-----------------------+------------------+-------------------+
|Adams-Normandie        |8791.458301453711 |0.7148686559551135 |
|Alsace                 |11239.50136425648 |0.5416098226466576 |
|Angeles National Forest|33079.58823529412 |0.4117647058823529 |
|Angelino Heights       |18427.059814658805|0.5732940185341197 |
|Arleta                 |12110.779474388612|0.4264509064363061 |
|Atwater Village        |28481.236967160792|0.5288318320448259 |
|Baldwin Hills          |17303.906408241663|0.9950061114021302 |
|Bel Air                |63041.33942621959 |0.39922527539038855|
|Beverly Crest          |60947.48978754819 |0.3689607087195472 |
|Beverlywood            |29267.82118405155 |0.5084977849375755 |
|Boyle Heights          |8494.108286861341 |0.6171887393378809 |
|Brentwood              |60846.854461055365|0.4058638814936173 |
|Brookside              |

In [20]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 22.81 seconds

In [21]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (35)
+- Sort (34)
   +- Exchange (33)
      +- Project (32)
         +- HashAggregate (31)
            +- Exchange (30)
               +- HashAggregate (29)
                  +- Project (28)
                     +- RangeJoin (27)
                        :- Filter (19)
                        :  +- ObjectHashAggregate (18)
                        :     +- Exchange (17)
                        :        +- ObjectHashAggregate (16)
                        :           +- Project (15)
                        :              +- ShuffledHashJoin Inner BuildLeft (14)
                        :                 :- Exchange (9)
                        :                 :  +- ObjectHashAggregate (8)
                        :                 :     +- Exchange (7)
                        :                 :        +- ObjectHashAggregate (6)
                        :                 :           +- Project (5)
                        :                 :              

### 4. SHUFFLE REPLICATE NL

In [22]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Για κάθε περιοχή του στο Census, υπάρχουν πολλαπλές
# εγγραφές για COMM, ZCTA10. Συνεπώς, πριν κάνουμε το
# join με το Income, πρέπει να κάνουμε group by για να
# υπολογίσουμε το συνολικό πληθυσμό ανά COMM, ZIP Code
result = blocks_data \
            .groupBy("COMM", "ZCTA10") \
            .agg(
                sum("POP_2010").alias("Population"),
                sum("HOUSING10").alias("Households"),
                ST_Union_Aggr("geometry").alias("geometry")
            )

# Έπειτα κάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code. Κάνουμε group by με το
# COMM για να βρούμε το συνολικό εισόδημα και το
# συνολικό πληθυσμό ανά COMM.
result = result \
            .hint("SHUFFLE_REPLICATE_NL") \
            .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            )

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = result \
            .hint("SHUFFLE_REPLICATE_NL") \
            .join(crime_data, ST_Within(crime_data["geom"], result["geometry"]), "inner") \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .orderBy("COMM")

result \
    .select("COMM", "Income per Person", "Crimes per Person") \
    .show(truncate=False)

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------------------+-------------------+
|COMM                   |Income per Person |Crimes per Person  |
+-----------------------+------------------+-------------------+
|Adams-Normandie        |8791.458301453711 |0.7148686559551135 |
|Alsace                 |11239.50136425648 |0.5416098226466576 |
|Angeles National Forest|33079.58823529412 |0.4117647058823529 |
|Angelino Heights       |18427.059814658805|0.5732940185341197 |
|Arleta                 |12110.779474388612|0.4264509064363061 |
|Atwater Village        |28481.236967160792|0.5288318320448259 |
|Baldwin Hills          |17303.906408241663|0.9950061114021302 |
|Bel Air                |63041.33942621959 |0.39922527539038855|
|Beverly Crest          |60947.48978754819 |0.3689607087195472 |
|Beverlywood            |29267.82118405155 |0.5084977849375755 |
|Boyle Heights          |8494.108286861341 |0.6171887393378809 |
|Brentwood              |60846.854461055365|0.4058638814936173 |
|Brookside              |

In [23]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 16.40 seconds

In [24]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (33)
+- Sort (32)
   +- Exchange (31)
      +- Project (30)
         +- HashAggregate (29)
            +- Exchange (28)
               +- HashAggregate (27)
                  +- Project (26)
                     +- RangeJoin (25)
                        :- Filter (17)
                        :  +- ObjectHashAggregate (16)
                        :     +- Exchange (15)
                        :        +- ObjectHashAggregate (14)
                        :           +- Project (13)
                        :              +- CartesianProduct Inner (12)
                        :                 :- ObjectHashAggregate (8)
                        :                 :  +- Exchange (7)
                        :                 :     +- ObjectHashAggregate (6)
                        :                 :        +- Project (5)
                        :                 :           +- Filter (4)
                        :                 :              +- Generate 

## Συμπεράσματα

Η στρατηγική join που πετυχαίνει την καλύτερη επίδοση είναι η **SHUFFLE REPLICATE NL** (*Shuffle Replicate Nested Loop*) με χρόνο 16.40 δευτερόλεπτα.

Η δεύτερη καλύτερη στρατηγική με λίγο μεγαλύτερο χρόνο (που μπορεί να οφείλεται και στην τυχαιότητα) είναι η **SHUFFLE HASH** με 22.81 δευτερόλεπτα, ενώ οι στρατηγικές **MERGE** και **BROADCAST** είναι λίγο πιο αργές, με χρόνους 25.10 και 26.90 δευτερόλεπτα αντίστοιχα.

Συνεπώς η καταλληλότερη στρατηγική για την περίπτωσή μας είναι η **SHUFFLE_REPLICATE_NL**.